In this notebook we will match the mask numbers of the nuclei and the cell body. 
In order to do this, the segmentation notebook should have been run with struct = CELL and
with struct = NUCLEI.

Jacques Bourg @ Florian Muller lab. Institut Pasteur.

<img src="./pipeline.png" alt="pipeline" width="1200" height="477">

In [ ]:
import sys
import os
import numpy as np
import napari
from skimage import io
from pathlib import Path
import ipywidgets as widgets
from IPython.display import display

In [ ]:
%load_ext autoreload
%autoreload 2

base_dir = Path("../../src").resolve()
sys.path.append(str(base_dir))
sys.path.append(str(base_dir / "utils"))
sys.path.append(str(base_dir / "segmentation"))
sys.path.append(str(base_dir / "synthesis"))

from segmentation.refine_seg import Segmentation
from synthesis.synthesize import Synthesis
from utils.parameters_tracking import Parameter_tracking as Track
from utils.file_handling import FileProcessor

sg  = Segmentation()
st  = Synthesis()
tk  = Track()
fp  = FileProcessor()

In [ ]:
var = str(Path('../Analysis'))
batch_folders = os.listdir(var)
dropdown = widgets.Dropdown(options=batch_folders, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown)

In [ ]:
n         = np.where(np.array(batch_folders) == dropdown.value)[0][0]
file_path = str(Path(var) / Path(batch_folders[n]) / Path(batch_folders[n] +'.json'))
constants = tk.load_json(file_path)
batch_name= constants['BATCH_NAME']; print(batch_name)

In [ ]:
modalities = constants['MODALITIES']
dropdown2 = widgets.Dropdown(options=modalities, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown2)

In [ ]:
n2  = np.where(np.array(modalities) == dropdown2.value)[0][0] 
modality = modalities[n2]; print(modality)

First give the channel used for segmenting the cell body (and confirm it is the cell body)

In [ ]:
channel   = constants['CHANNELS']
dropdown3 = widgets.Dropdown(options=channel, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown3)

In [ ]:
n3   = np.where(np.array(channel) == dropdown3.value)[0][0]
chan_cell = channel[n3]; print(chan_cell)

Choose the structure refering to the cell body

In [ ]:
structs   = constants['STRUCTURES']
dropdown4 = widgets.Dropdown(options=structs, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown4)

In [ ]:
n4   = np.where(np.array(structs) == dropdown4.value)[0][0]
struc_cell = structs[n4]; print(struc_cell)

In [ ]:
# load masks associated with the cells
path_mask_cell  = constants[f"MASK_{modality}_{chan_cell}_{struc_cell}_LIST_PATH"]
dict_mask_cells = fp.load_masks_distributed_files(path_mask_cell)

Give the channel for segmenting the nuclei (and confirm it is the cell nuclei)

In [ ]:
channel   = constants['CHANNELS']
dropdown5 = widgets.Dropdown(options=channel, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown5)

In [ ]:
n5       = np.where(np.array(channel) == dropdown5.value)[0][0]
chan_nuc = channel[n5]; print(chan_nuc)

Choose the structure name associated with the nucleus.

In [ ]:
structs   = constants['STRUCTURES']
dropdown6 = widgets.Dropdown(options=structs, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown6)

In [ ]:
n6   = np.where(np.array(structs) == dropdown6.value)[0][0]
struc_nuc = structs[n6]; print(struc_nuc)

In [ ]:
# load masks associated with the nuclei
path_mask_nuclei = constants[f"MASK_{modality}_{chan_nuc}_{struc_nuc}_LIST_PATH"]
dict_mask_nuc    = fp.load_masks_distributed_files(path_mask_nuclei)

In [ ]:
# create folder for segmentations of cells and nuclei
folder_path_seg = Path(f"../Analysis/{batch_name}/{modality}/segmentations_nuclei_cell_matched")
if not folder_path_seg.exists():
    folder_path_seg.mkdir(parents=True)
    
folder_path_seg_nuclei = Path(f"../Analysis/{batch_name}/{modality}/segmentations_nuclei_cell_matched/{struc_nuc}")
if not folder_path_seg_nuclei.exists():
    folder_path_seg_nuclei.mkdir(parents=True)    
    
folder_path_seg_cell = Path(f"../Analysis/{batch_name}/{modality}/segmentations_nuclei_cell_matched/{struc_cell}")
if not folder_path_seg_cell.exists():
    folder_path_seg_cell.mkdir(parents=True)

In [ ]:
dict_mask_nuc_matched           = {}
dict_mask_cells_matched         = {}
mask_nuc_matched_path           = Path(f"../Analysis/{batch_name}/{modality}/segmentations_nuclei_cell_matched/{struc_nuc}/{batch_name}_masks_nuc_matched.npy")
mask_cell_matched_path          = Path(f"../Analysis/{batch_name}/{modality}/segmentations_nuclei_cell_matched/{struc_cell}/{batch_name}_masks_cells_matched.npy")

dict_contours_nuc_matched       = {}
dict_contours_cells_matched     = {}
mask_nuc_matched_contours_path  = Path(f"../Analysis/{batch_name}/{modality}/segmentations_nuclei_cell_matched/{struc_nuc}/{batch_name}_contours_nuc_matched.npy")
mask_cell_matched_contours_path = Path(f"../Analysis/{batch_name}/{modality}/segmentations_nuclei_cell_matched/{struc_cell}/{batch_name}_contours_cells_matched.npy")

batch_nuc  = constants[f'BATCH_{modality}_{chan_nuc}']
batch_cell = constants[f'BATCH_{modality}_{chan_cell}']

viewer_match = napari.Viewer(title="Matching masks of cells an nuclei")
counter = 0
for ind_l in range(len(batch_nuc)):
        file_add  = batch_nuc[ind_l]
        file_base = '_'.join(Path(file_add).stem.split('_')[:-1])

        im        = io.imread(file_add)
        im_nuc    = np.max(im, axis=0) 
        val_im_nuc= np.percentile(im_nuc, 99)
        
        im         = io.imread(batch_cell[ind_l])
        im_cell    = np.max(im, axis=0) 
        val_im_cell= np.percentile(im_cell, 99)
        
        masks_nuc   = dict_mask_nuc[file_base]
        masks_cells = dict_mask_cells[file_base]

        masks_nuc_matched, masks_cells_matched = sg.match_nuc_cell(masks_nuc.copy(), masks_cells.copy())
        dict_mask_nuc_matched[file_base]   = masks_nuc_matched
        dict_mask_cells_matched[file_base] = masks_cells_matched

        dict_contours_nuc_matched[file_base]   = sg.find_all_contours(masks_nuc_matched)
        dict_contours_cells_matched[file_base] = sg.find_all_contours(masks_cells_matched)

        viewer_match.add_image(im_nuc, rgb=False, name=f"DAPI {file_base}", contrast_limits=(0, val_im_nuc), opacity=0.5, colormap='red', blending='additive')
        viewer_match.add_image(im_cell, rgb=False, name=f"FISH {file_base}", contrast_limits=(0, val_im_cell), opacity=0.5, colormap='green', blending='additive')

        viewer_match.add_labels(masks_cells, name=f"Old mask cells {file_base}", opacity=.8, blending='translucent')
        viewer_match.add_labels(masks_nuc, name=f"Old mask nuclei {file_base}", opacity=.8, blending='translucent')

        viewer_match.add_labels(masks_cells_matched, name=f"Mask cells {file_base}", opacity=.8, blending='translucent')
        viewer_match.add_labels(masks_nuc_matched, name=f"Mask nuclei {file_base}", opacity=.8, blending='translucent')

        if counter != 0:
            viewer_match.layers[f"DAPI {file_base}"].visible            = False
            viewer_match.layers[f"FISH {file_base}"].visible            = False
            viewer_match.layers[f"Mask cells {file_base}"].visible      = False
            viewer_match.layers[f"Mask nuclei {file_base}"].visible     = False
            viewer_match.layers[f"Old mask cells {file_base}"].visible  = False
            viewer_match.layers[f"Old mask nuclei {file_base}"].visible = False
        counter +=1    
  
fp.save_masks_distributed_files(mask_nuc_matched_path, dict_mask_nuc_matched)  
fp.save_masks_distributed_files(mask_cell_matched_path, dict_mask_cells_matched)  

np.save(mask_nuc_matched_contours_path, dict_contours_nuc_matched)
np.save(mask_cell_matched_contours_path, dict_contours_cells_matched)

Choose the detection method for doing the spot detection (and then the stats per cell).

In [ ]:
detection_methods = ['UFISH','BIGFISH']
dropdown8         = widgets.Dropdown(options=detection_methods, description='Select:', layout=widgets.Layout(width='auto', min_width='150px'))
display(dropdown8)

In [ ]:
n8     = np.where(np.array(detection_methods) == dropdown8.value)[0][0]
meth   = detection_methods[n8]; print(meth)

In [ ]:
stats_folder = Path(f"../Analysis/{batch_name}/{modality}/segmentations_nuclei_cell_matched/stats")    # create the folder masks stats (for the cells)
if not stats_folder.exists():
    stats_folder.mkdir(parents=True)
print(stats_folder)     

stats_folder_meth = Path(f"../Analysis/{batch_name}/{modality}/segmentations_nuclei_cell_matched/stats/{meth}")    # create the folder masks stats (for the cells)
if not stats_folder_meth.exists():
    stats_folder_meth.mkdir(parents=True)
print(stats_folder_meth)

for chan_f_t in constants['CHANNELS']:
    if chan_f_t != 'DAPI':
        stats_gene_folder_meth = Path(f"../Analysis/{batch_name}/{modality}/segmentations_nuclei_cell_matched/stats/{meth}/{chan_f_t}")    # create the folder masks stats (for the cells)
        if not stats_gene_folder_meth.exists():
            stats_gene_folder_meth.mkdir(parents=True)
        print(stats_gene_folder_meth)

        file_ref_mask_stats = stats_gene_folder_meth / f'masks_stats_refs.npy'
        fp.save_masks_stats_distributed_files_init(file_ref_mask_stats, mask_nuc_matched_path, col_name='Cell_ID')  # numpy_file_add: str,  mask_path: str, col_name='Cell_ID'  
        
        exec(f"MASK_{modality}_{struc_cell}_STATS_{meth}_{chan_f_t}_LIST_PATH = file_ref_mask_stats", globals())

In [ ]:
exec(f"MASK_{modality}_{struc_nuc}_LIST_PATH = mask_nuc_matched_path", globals())
exec(f"MASK_{modality}_{struc_cell}_LIST_PATH = mask_cell_matched_path", globals())

exec(f"MASK_{modality}_{struc_nuc}_CONTOURS_PATH = mask_nuc_matched_contours_path", globals())
exec(f"MASK_{modality}_{struc_cell}_CONTOURS_PATH = mask_cell_matched_contours_path", globals())

In [ ]:
constants2 = tk.collect_constants()
tk.save_constants_and_commit_hash(constants2, batch_name, folder_path = Path(f"../Analysis/{batch_name}"))